In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorflow.python.framework import ops

# Disable eager execution
tf.compat.v1.disable_eager_execution()

# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize and reshape input data
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(-1, 784)  # Flattening 28x28 images
x_test = x_test.reshape(-1, 784)

# One-hot encoding for labels
y_train_onehot = np.eye(10)[y_train]
y_test_onehot = np.eye(10)[y_test]

# Reset default graph
ops.reset_default_graph()

# Define placeholders
X = tf.compat.v1.placeholder(tf.float32, [None, 784])
Y = tf.compat.v1.placeholder(tf.float32, [None, 10])

# Define model parameters
def init_weights(shape):
    return tf.Variable(tf.random.normal(shape))

hidden_layer_sizes = [256, 128, 64]
activation_functions = [tf.nn.relu, tf.nn.sigmoid, tf.nn.tanh]

results = []  # Store results for table

def forward_propagation(X, num_neurons, activation):
    W = init_weights([784, num_neurons])
    b = tf.Variable(tf.zeros([num_neurons]))
    z = tf.matmul(X, W) + b
    a = activation(z)

    W_out = init_weights([num_neurons, 10])
    b_out = tf.Variable(tf.zeros([10]))
    logits = tf.matmul(a, W_out) + b_out  # No softmax here

    return logits

for num_neurons in hidden_layer_sizes:
    for activation in activation_functions:
        print(f"Training model with {num_neurons} neurons and {activation.__name__} activation")
        logits = forward_propagation(X, num_neurons, activation)

        # Loss function
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))

        # Optimizer
        learning_rate = 0.1
        optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(loss)

        # Accuracy metric
        correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

        # Training
        epochs = 50
        batch_size = 10
        loss_history = []
        accuracy_history = []

        with tf.compat.v1.Session() as sess:
            sess.run(tf.compat.v1.global_variables_initializer())
            start_time = time.time()

            for epoch in range(epochs):
                for i in range(0, x_train.shape[0], batch_size):
                    batch_x = x_train[i:i+batch_size]
                    batch_y = y_train_onehot[i:i+batch_size]
                    sess.run(optimizer, feed_dict={X: batch_x, Y: batch_y})

                train_loss, train_acc = sess.run([loss, accuracy], feed_dict={X: x_train, Y: y_train_onehot})
                loss_history.append(train_loss)
                accuracy_history.append(train_acc)
                print(f"Epoch {epoch+1}, Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

            training_time = time.time() - start_time

            # Evaluate on test set
            test_acc, test_predictions = sess.run([accuracy, tf.argmax(logits, 1)], feed_dict={X: x_test, Y: y_test_onehot})
            print(f"\nTest Accuracy: {test_acc:.4f}, Training Time: {training_time:.2f} seconds\n")

            # Confusion Matrix
            cm = confusion_matrix(y_test, test_predictions)

            # Store results
            results.append({
                "Neurons": num_neurons,
                "Activation": activation.__name__,
                "Test Accuracy": test_acc,
                "Training Time (s)": training_time,
                "Loss Curve": loss_history,
                "Accuracy Curve": accuracy_history,
                "Confusion Matrix": cm
            })

            # Plot loss curve
            plt.figure()
            plt.plot(range(1, epochs + 1), loss_history, label='Loss')
            plt.xlabel('Epochs')
            plt.ylabel('Loss')
            plt.title(f'Loss Curve ({num_neurons} Neurons, {activation.__name__})')
            plt.legend()
            plt.show()

            # Plot accuracy curve
            plt.figure()
            plt.plot(range(1, epochs + 1), accuracy_history, label='Accuracy', color='green')
            plt.xlabel('Epochs')
            plt.ylabel('Accuracy')
            plt.title(f'Accuracy Curve ({num_neurons} Neurons, {activation.__name__})')
            plt.legend()
            plt.show()

# Convert results to DataFrame and print table
results_df = pd.DataFrame(results, columns=["Neurons", "Activation", "Test Accuracy", "Training Time (s)", "Loss Curve", "Accuracy Curve", "Confusion Matrix"])
print(results_df)

# Plot Confusion Matrices
for index, row in results_df.iterrows():
    plt.figure(figsize=(6,6))
    sns.heatmap(row["Confusion Matrix"], annot=True, fmt='d', cmap='Blues', xticklabels=range(10), yticklabels=range(10))
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix ({row["Neurons"]} Neurons, {row["Activation"]})')
    plt.show()


In [ ]:
Downloading data from https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training model with 256 neurons and relu activation
Epoch 1, Loss: 0.4188, Accuracy: 0.9148
Epoch 2, Loss: 0.2500, Accuracy: 0.9403
Epoch 3, Loss: 0.1706, Accuracy: 0.9546
Epoch 4, Loss: 0.1417, Accuracy: 0.9597
Epoch 5, Loss: 0.1174, Accuracy: 0.9652
Epoch 6, Loss: 0.1054, Accuracy: 0.9685
Epoch 7, Loss: 0.0894, Accuracy: 0.9725
Epoch 8, Loss: 0.0850, Accuracy: 0.9744
Epoch 9, Loss: 0.0695, Accuracy: 0.9784
Epoch 10, Loss: 0.0607, Accuracy: 0.9806
Epoch 11, Loss: 0.0532, Accuracy: 0.9825
Epoch 12, Loss: 0.0441, Accuracy: 0.9850
Epoch 13, Loss: 0.0393, Accuracy: 0.9867
Epoch 14, Loss: 0.0344, Accuracy: 0.9885
Epoch 15, Loss: 0.0298, Accuracy: 0.9902
Epoch 16, Loss: 0.0271, Accuracy: 0.9909
Epoch 17, Loss: 0.0243, Accuracy: 0.9918
Epoch 18, Loss: 0.0221, Accuracy: 0.9928
Epoch 19, Loss: 0.0192, Accuracy: 0.9936
Epoch 20, Loss: 0.0165, Accuracy: 0.9948
Epoch 21, Loss: 0.0151, Accuracy: 0.9953
Epoch 22, Loss: 0.0131, Accuracy: 0.9959
Epoch 23, Loss: 0.0115, Accuracy: 0.9965
Epoch 24, Loss: 0.0107, Accuracy: 0.9969
Epoch 25, Loss: 0.0085, Accuracy: 0.9977
Epoch 26, Loss: 0.0074, Accuracy: 0.9981
Epoch 27, Loss: 0.0067, Accuracy: 0.9983
Epoch 28, Loss: 0.0061, Accuracy: 0.9984
Epoch 29, Loss: 0.0055, Accuracy: 0.9987
Epoch 30, Loss: 0.0052, Accuracy: 0.9987
Epoch 31, Loss: 0.0048, Accuracy: 0.9989
Epoch 32, Loss: 0.0044, Accuracy: 0.9990
Epoch 33, Loss: 0.0042, Accuracy: 0.9990
Epoch 34, Loss: 0.0040, Accuracy: 0.9990
Epoch 35, Loss: 0.0038, Accuracy: 0.9992
Epoch 36, Loss: 0.0036, Accuracy: 0.9992
Epoch 37, Loss: 0.0034, Accuracy: 0.9994
Epoch 38, Loss: 0.0032, Accuracy: 0.9994
Epoch 39, Loss: 0.0031, Accuracy: 0.9994
Epoch 40, Loss: 0.0030, Accuracy: 0.9995
Epoch 41, Loss: 0.0028, Accuracy: 0.9995
Epoch 42, Loss: 0.0026, Accuracy: 0.9995
Epoch 43, Loss: 0.0025, Accuracy: 0.9995
Epoch 44, Loss: 0.0024, Accuracy: 0.9995
Epoch 45, Loss: 0.0023, Accuracy: 0.9995
Epoch 46, Loss: 0.0022, Accuracy: 0.9996
Epoch 47, Loss: 0.0021, Accuracy: 0.9996
Epoch 48, Loss: 0.0020, Accuracy: 0.9996
Epoch 49, Loss: 0.0019, Accuracy: 0.9996
Epoch 50, Loss: 0.0018, Accuracy: 0.9997

Test Accuracy: 0.9603, Training Time: 356.81 seconds



Training model with 256 neurons and sigmoid activation
Epoch 1, Loss: 0.3540, Accuracy: 0.9116
Epoch 2, Loss: 0.2407, Accuracy: 0.9338
Epoch 3, Loss: 0.1878, Accuracy: 0.9457
Epoch 4, Loss: 0.1584, Accuracy: 0.9519
Epoch 5, Loss: 0.1381, Accuracy: 0.9569
Epoch 6, Loss: 0.1224, Accuracy: 0.9611
Epoch 7, Loss: 0.1094, Accuracy: 0.9649
Epoch 8, Loss: 0.0982, Accuracy: 0.9681
Epoch 9, Loss: 0.0887, Accuracy: 0.9714
Epoch 10, Loss: 0.0805, Accuracy: 0.9739
Epoch 11, Loss: 0.0736, Accuracy: 0.9760
Epoch 12, Loss: 0.0677, Accuracy: 0.9782
Epoch 13, Loss: 0.0627, Accuracy: 0.9794
Epoch 14, Loss: 0.0584, Accuracy: 0.9809
Epoch 15, Loss: 0.0545, Accuracy: 0.9824
Epoch 16, Loss: 0.0510, Accuracy: 0.9834
Epoch 17, Loss: 0.0478, Accuracy: 0.9847
Epoch 18, Loss: 0.0448, Accuracy: 0.9857
Epoch 19, Loss: 0.0420, Accuracy: 0.9863
Epoch 20, Loss: 0.0392, Accuracy: 0.9873
Epoch 21, Loss: 0.0366, Accuracy: 0.9883
Epoch 22, Loss: 0.0341, Accuracy: 0.9894
Epoch 23, Loss: 0.0318, Accuracy: 0.9903
Epoch 24, Loss: 0.0296, Accuracy: 0.9912
Epoch 25, Loss: 0.0275, Accuracy: 0.9919
Epoch 26, Loss: 0.0256, Accuracy: 0.9927
Epoch 27, Loss: 0.0238, Accuracy: 0.9933
Epoch 28, Loss: 0.0222, Accuracy: 0.9939
Epoch 29, Loss: 0.0207, Accuracy: 0.9945
Epoch 30, Loss: 0.0193, Accuracy: 0.9952
Epoch 31, Loss: 0.0181, Accuracy: 0.9955
Epoch 32, Loss: 0.0169, Accuracy: 0.9959
Epoch 33, Loss: 0.0158, Accuracy: 0.9964
Epoch 34, Loss: 0.0148, Accuracy: 0.9969
Epoch 35, Loss: 0.0139, Accuracy: 0.9973
Epoch 36, Loss: 0.0131, Accuracy: 0.9976
Epoch 37, Loss: 0.0123, Accuracy: 0.9980
Epoch 38, Loss: 0.0116, Accuracy: 0.9984
Epoch 39, Loss: 0.0109, Accuracy: 0.9986
Epoch 40, Loss: 0.0103, Accuracy: 0.9988
Epoch 41, Loss: 0.0098, Accuracy: 0.9988
Epoch 42, Loss: 0.0093, Accuracy: 0.9990
Epoch 43, Loss: 0.0089, Accuracy: 0.9991
Epoch 44, Loss: 0.0085, Accuracy: 0.9991
Epoch 45, Loss: 0.0081, Accuracy: 0.9992
Epoch 46, Loss: 0.0078, Accuracy: 0.9993
Epoch 47, Loss: 0.0075, Accuracy: 0.9994
Epoch 48, Loss: 0.0072, Accuracy: 0.9995
Epoch 49, Loss: 0.0069, Accuracy: 0.9995
Epoch 50, Loss: 0.0066, Accuracy: 0.9997

Test Accuracy: 0.9566, Training Time: 353.33 seconds



Training model with 256 neurons and tanh activation
Epoch 1, Loss: 0.5016, Accuracy: 0.9081
Epoch 2, Loss: 0.2970, Accuracy: 0.9317
Epoch 3, Loss: 0.2103, Accuracy: 0.9450
Epoch 4, Loss: 0.1740, Accuracy: 0.9511
Epoch 5, Loss: 0.1562, Accuracy: 0.9535
Epoch 6, Loss: 0.1321, Accuracy: 0.9590
Epoch 7, Loss: 0.1187, Accuracy: 0.9623
Epoch 8, Loss: 0.1024, Accuracy: 0.9664
Epoch 9, Loss: 0.0886, Accuracy: 0.9700
Epoch 10, Loss: 0.0797, Accuracy: 0.9733
Epoch 11, Loss: 0.0714, Accuracy: 0.9752
Epoch 12, Loss: 0.0648, Accuracy: 0.9776
Epoch 13, Loss: 0.0578, Accuracy: 0.9794
Epoch 14, Loss: 0.0518, Accuracy: 0.9812
Epoch 15, Loss: 0.0450, Accuracy: 0.9836
Epoch 16, Loss: 0.0391, Accuracy: 0.9855
Epoch 17, Loss: 0.0340, Accuracy: 0.9873
Epoch 18, Loss: 0.0300, Accuracy: 0.9889
Epoch 19, Loss: 0.0263, Accuracy: 0.9909
Epoch 20, Loss: 0.0231, Accuracy: 0.9921
Epoch 21, Loss: 0.0202, Accuracy: 0.9934
Epoch 22, Loss: 0.0180, Accuracy: 0.9943
Epoch 23, Loss: 0.0162, Accuracy: 0.9952
Epoch 24, Loss: 0.0146, Accuracy: 0.9958
Epoch 25, Loss: 0.0134, Accuracy: 0.9963
Epoch 26, Loss: 0.0124, Accuracy: 0.9966
Epoch 27, Loss: 0.0115, Accuracy: 0.9971
Epoch 28, Loss: 0.0107, Accuracy: 0.9973
Epoch 29, Loss: 0.0100, Accuracy: 0.9974
Epoch 30, Loss: 0.0094, Accuracy: 0.9977
Epoch 31, Loss: 0.0089, Accuracy: 0.9979
Epoch 32, Loss: 0.0084, Accuracy: 0.9980
Epoch 33, Loss: 0.0079, Accuracy: 0.9983
Epoch 34, Loss: 0.0074, Accuracy: 0.9985
Epoch 35, Loss: 0.0069, Accuracy: 0.9987
Epoch 36, Loss: 0.0063, Accuracy: 0.9990
Epoch 37, Loss: 0.0058, Accuracy: 0.9992
Epoch 38, Loss: 0.0053, Accuracy: 0.9994
Epoch 39, Loss: 0.0049, Accuracy: 0.9994
Epoch 40, Loss: 0.0045, Accuracy: 0.9995
Epoch 41, Loss: 0.0042, Accuracy: 0.9995
Epoch 42, Loss: 0.0039, Accuracy: 0.9997
Epoch 43, Loss: 0.0035, Accuracy: 0.9998
Epoch 44, Loss: 0.0033, Accuracy: 0.9998
Epoch 45, Loss: 0.0030, Accuracy: 0.9998
Epoch 46, Loss: 0.0028, Accuracy: 0.9999
Epoch 47, Loss: 0.0026, Accuracy: 0.9999
Epoch 48, Loss: 0.0025, Accuracy: 0.9999
Epoch 49, Loss: 0.0024, Accuracy: 0.9999
Epoch 50, Loss: 0.0023, Accuracy: 0.9999

Test Accuracy: 0.9453, Training Time: 346.11 seconds



Training model with 128 neurons and relu activation
Epoch 1, Loss: 0.3933, Accuracy: 0.8971
Epoch 2, Loss: 0.3030, Accuracy: 0.9168
Epoch 3, Loss: 0.2493, Accuracy: 0.9280
Epoch 4, Loss: 0.2139, Accuracy: 0.9369
Epoch 5, Loss: 0.1897, Accuracy: 0.9430
Epoch 6, Loss: 0.1744, Accuracy: 0.9481
Epoch 7, Loss: 0.1612, Accuracy: 0.9526
Epoch 8, Loss: 0.1485, Accuracy: 0.9564
Epoch 9, Loss: 0.1344, Accuracy: 0.9598
Epoch 10, Loss: 0.1255, Accuracy: 0.9614
Epoch 11, Loss: 0.1184, Accuracy: 0.9639
Epoch 12, Loss: 0.1114, Accuracy: 0.9666
Epoch 13, Loss: 0.1061, Accuracy: 0.9679
Epoch 14, Loss: 0.1005, Accuracy: 0.9698
Epoch 15, Loss: 0.0931, Accuracy: 0.9721
Epoch 16, Loss: 0.0874, Accuracy: 0.9739
Epoch 17, Loss: 0.0836, Accuracy: 0.9751
Epoch 18, Loss: 0.0819, Accuracy: 0.9753
Epoch 19, Loss: 0.0773, Accuracy: 0.9765
Epoch 20, Loss: 0.0746, Accuracy: 0.9772
Epoch 21, Loss: 0.0701, Accuracy: 0.9787
Epoch 22, Loss: 0.0671, Accuracy: 0.9792
Epoch 23, Loss: 0.0650, Accuracy: 0.9801
Epoch 24, Loss: 0.0623, Accuracy: 0.9809
Epoch 25, Loss: 0.0571, Accuracy: 0.9825
Epoch 26, Loss: 0.0603, Accuracy: 0.9816
Epoch 27, Loss: 0.0517, Accuracy: 0.9839
Epoch 28, Loss: 0.0499, Accuracy: 0.9842
Epoch 29, Loss: 0.0502, Accuracy: 0.9838
Epoch 30, Loss: 0.0453, Accuracy: 0.9856
Epoch 31, Loss: 0.0416, Accuracy: 0.9863
Epoch 32, Loss: 0.0411, Accuracy: 0.9868
Epoch 33, Loss: 0.0381, Accuracy: 0.9877
Epoch 34, Loss: 0.0355, Accuracy: 0.9886
Epoch 35, Loss: 0.0342, Accuracy: 0.9887
Epoch 36, Loss: 0.0322, Accuracy: 0.9897
Epoch 37, Loss: 0.0313, Accuracy: 0.9897
Epoch 38, Loss: 0.0291, Accuracy: 0.9905
Epoch 39, Loss: 0.0268, Accuracy: 0.9911
Epoch 40, Loss: 0.0257, Accuracy: 0.9914
Epoch 41, Loss: 0.0243, Accuracy: 0.9919
Epoch 42, Loss: 0.0227, Accuracy: 0.9924
Epoch 43, Loss: 0.0214, Accuracy: 0.9927
Epoch 44, Loss: 0.0192, Accuracy: 0.9935
Epoch 45, Loss: 0.0183, Accuracy: 0.9941
Epoch 46, Loss: 0.0174, Accuracy: 0.9944
Epoch 47, Loss: 0.0161, Accuracy: 0.9947
Epoch 48, Loss: 0.0157, Accuracy: 0.9947
Epoch 49, Loss: 0.0147, Accuracy: 0.9952
Epoch 50, Loss: 0.0141, Accuracy: 0.9954

Test Accuracy: 0.9541, Training Time: 354.56 seconds



Training model with 128 neurons and sigmoid activation
Epoch 1, Loss: 0.3540, Accuracy: 0.8983
Epoch 2, Loss: 0.2611, Accuracy: 0.9218
Epoch 3, Loss: 0.2186, Accuracy: 0.9348
Epoch 4, Loss: 0.1919, Accuracy: 0.9426
Epoch 5, Loss: 0.1723, Accuracy: 0.9489
Epoch 6, Loss: 0.1568, Accuracy: 0.9536
Epoch 7, Loss: 0.1441, Accuracy: 0.9571
Epoch 8, Loss: 0.1334, Accuracy: 0.9599
Epoch 9, Loss: 0.1240, Accuracy: 0.9625
Epoch 10, Loss: 0.1156, Accuracy: 0.9652
Epoch 11, Loss: 0.1081, Accuracy: 0.9674
Epoch 12, Loss: 0.1014, Accuracy: 0.9694
Epoch 13, Loss: 0.0954, Accuracy: 0.9715
Epoch 14, Loss: 0.0900, Accuracy: 0.9733
Epoch 15, Loss: 0.0851, Accuracy: 0.9749
Epoch 16, Loss: 0.0806, Accuracy: 0.9761
Epoch 17, Loss: 0.0765, Accuracy: 0.9774
Epoch 18, Loss: 0.0727, Accuracy: 0.9786
Epoch 19, Loss: 0.0691, Accuracy: 0.9797
Epoch 20, Loss: 0.0658, Accuracy: 0.9807
Epoch 21, Loss: 0.0627, Accuracy: 0.9816
Epoch 22, Loss: 0.0598, Accuracy: 0.9825
Epoch 23, Loss: 0.0571, Accuracy: 0.9835
Epoch 24, Loss: 0.0545, Accuracy: 0.9842
Epoch 25, Loss: 0.0520, Accuracy: 0.9850
Epoch 26, Loss: 0.0497, Accuracy: 0.9856
Epoch 27, Loss: 0.0474, Accuracy: 0.9863
Epoch 28, Loss: 0.0453, Accuracy: 0.9870
Epoch 29, Loss: 0.0433, Accuracy: 0.9879
Epoch 30, Loss: 0.0414, Accuracy: 0.9884
Epoch 31, Loss: 0.0396, Accuracy: 0.9889
Epoch 32, Loss: 0.0379, Accuracy: 0.9894
Epoch 33, Loss: 0.0363, Accuracy: 0.9901
Epoch 34, Loss: 0.0347, Accuracy: 0.9906
Epoch 35, Loss: 0.0332, Accuracy: 0.9912
Epoch 36, Loss: 0.0317, Accuracy: 0.9916
Epoch 37, Loss: 0.0303, Accuracy: 0.9922
Epoch 38, Loss: 0.0289, Accuracy: 0.9927
Epoch 39, Loss: 0.0276, Accuracy: 0.9931
Epoch 40, Loss: 0.0263, Accuracy: 0.9936
Epoch 41, Loss: 0.0250, Accuracy: 0.9941
Epoch 42, Loss: 0.0239, Accuracy: 0.9946
Epoch 43, Loss: 0.0227, Accuracy: 0.9950
Epoch 44, Loss: 0.0216, Accuracy: 0.9954
Epoch 45, Loss: 0.0206, Accuracy: 0.9958
Epoch 46, Loss: 0.0196, Accuracy: 0.9961
Epoch 47, Loss: 0.0187, Accuracy: 0.9966
Epoch 48, Loss: 0.0179, Accuracy: 0.9969
Epoch 49, Loss: 0.0170, Accuracy: 0.9972
Epoch 50, Loss: 0.0163, Accuracy: 0.9973

Test Accuracy: 0.9588, Training Time: 353.85 seconds



Training model with 128 neurons and tanh activation
Epoch 1, Loss: 0.4428, Accuracy: 0.8773
Epoch 2, Loss: 0.3059, Accuracy: 0.9087
Epoch 3, Loss: 0.2389, Accuracy: 0.9273
Epoch 4, Loss: 0.2080, Accuracy: 0.9369
Epoch 5, Loss: 0.1828, Accuracy: 0.9445
Epoch 6, Loss: 0.1678, Accuracy: 0.9484
Epoch 7, Loss: 0.1513, Accuracy: 0.9535
Epoch 8, Loss: 0.1403, Accuracy: 0.9567
Epoch 9, Loss: 0.1310, Accuracy: 0.9593
Epoch 10, Loss: 0.1234, Accuracy: 0.9617
Epoch 11, Loss: 0.1177, Accuracy: 0.9630
Epoch 12, Loss: 0.1125, Accuracy: 0.9646
Epoch 13, Loss: 0.1065, Accuracy: 0.9661
Epoch 14, Loss: 0.1014, Accuracy: 0.9680
Epoch 15, Loss: 0.0965, Accuracy: 0.9695
Epoch 16, Loss: 0.0908, Accuracy: 0.9715
Epoch 17, Loss: 0.0860, Accuracy: 0.9727
Epoch 18, Loss: 0.0819, Accuracy: 0.9738
Epoch 19, Loss: 0.0781, Accuracy: 0.9751
Epoch 20, Loss: 0.0736, Accuracy: 0.9766
Epoch 21, Loss: 0.0693, Accuracy: 0.9784
Epoch 22, Loss: 0.0656, Accuracy: 0.9796
Epoch 23, Loss: 0.0623, Accuracy: 0.9808
Epoch 24, Loss: 0.0593, Accuracy: 0.9817
Epoch 25, Loss: 0.0566, Accuracy: 0.9824
Epoch 26, Loss: 0.0541, Accuracy: 0.9833
Epoch 27, Loss: 0.0516, Accuracy: 0.9841
Epoch 28, Loss: 0.0491, Accuracy: 0.9850
Epoch 29, Loss: 0.0469, Accuracy: 0.9857
Epoch 30, Loss: 0.0449, Accuracy: 0.9862
Epoch 31, Loss: 0.0431, Accuracy: 0.9869
Epoch 32, Loss: 0.0415, Accuracy: 0.9872
Epoch 33, Loss: 0.0399, Accuracy: 0.9875
Epoch 34, Loss: 0.0385, Accuracy: 0.9879
Epoch 35, Loss: 0.0370, Accuracy: 0.9885
Epoch 36, Loss: 0.0354, Accuracy: 0.9890
Epoch 37, Loss: 0.0338, Accuracy: 0.9897
Epoch 38, Loss: 0.0323, Accuracy: 0.9903
Epoch 39, Loss: 0.0308, Accuracy: 0.9909
Epoch 40, Loss: 0.0295, Accuracy: 0.9913
Epoch 41, Loss: 0.0282, Accuracy: 0.9918
Epoch 42, Loss: 0.0270, Accuracy: 0.9923
Epoch 43, Loss: 0.0259, Accuracy: 0.9926
Epoch 44, Loss: 0.0249, Accuracy: 0.9929
Epoch 45, Loss: 0.0238, Accuracy: 0.9934
Epoch 46, Loss: 0.0228, Accuracy: 0.9937
Epoch 47, Loss: 0.0217, Accuracy: 0.9940
Epoch 48, Loss: 0.0207, Accuracy: 0.9943
Epoch 49, Loss: 0.0197, Accuracy: 0.9947
Epoch 50, Loss: 0.0187, Accuracy: 0.9949

Test Accuracy: 0.9403, Training Time: 343.05 seconds



Training model with 64 neurons and relu activation
Epoch 1, Loss: 0.5167, Accuracy: 0.8605
Epoch 2, Loss: 0.3640, Accuracy: 0.9008
Epoch 3, Loss: 0.3131, Accuracy: 0.9135
Epoch 4, Loss: 0.2903, Accuracy: 0.9198
Epoch 5, Loss: 0.2533, Accuracy: 0.9288
Epoch 6, Loss: 0.2294, Accuracy: 0.9340
Epoch 7, Loss: 0.2108, Accuracy: 0.9388
Epoch 8, Loss: 0.1948, Accuracy: 0.9441
Epoch 9, Loss: 0.1790, Accuracy: 0.9484
Epoch 10, Loss: 0.1673, Accuracy: 0.9518
Epoch 11, Loss: 0.1525, Accuracy: 0.9556
Epoch 12, Loss: 0.1428, Accuracy: 0.9580
Epoch 13, Loss: 0.1358, Accuracy: 0.9598
Epoch 14, Loss: 0.1320, Accuracy: 0.9610
Epoch 15, Loss: 0.1284, Accuracy: 0.9617
Epoch 16, Loss: 0.1237, Accuracy: 0.9632
Epoch 17, Loss: 0.1190, Accuracy: 0.9645
Epoch 18, Loss: 0.1154, Accuracy: 0.9651
Epoch 19, Loss: 0.1111, Accuracy: 0.9664
Epoch 20, Loss: 0.1082, Accuracy: 0.9674
Epoch 21, Loss: 0.1046, Accuracy: 0.9684
Epoch 22, Loss: 0.1014, Accuracy: 0.9692
Epoch 23, Loss: 0.0988, Accuracy: 0.9702
Epoch 24, Loss: 0.0928, Accuracy: 0.9722
Epoch 25, Loss: 0.0908, Accuracy: 0.9722
Epoch 26, Loss: 0.0895, Accuracy: 0.9730
Epoch 27, Loss: 0.0846, Accuracy: 0.9745
Epoch 28, Loss: 0.0863, Accuracy: 0.9740
Epoch 29, Loss: 0.0826, Accuracy: 0.9748
Epoch 30, Loss: 0.0886, Accuracy: 0.9733
Epoch 31, Loss: 0.0815, Accuracy: 0.9754
Epoch 32, Loss: 0.0801, Accuracy: 0.9757
Epoch 33, Loss: 0.0825, Accuracy: 0.9755
Epoch 34, Loss: 0.0848, Accuracy: 0.9745
Epoch 35, Loss: 0.0800, Accuracy: 0.9758
Epoch 36, Loss: 0.0770, Accuracy: 0.9766
Epoch 37, Loss: 0.0758, Accuracy: 0.9769
Epoch 38, Loss: 0.0780, Accuracy: 0.9766
Epoch 39, Loss: 0.0706, Accuracy: 0.9784
Epoch 40, Loss: 0.0675, Accuracy: 0.9792
Epoch 41, Loss: 0.0675, Accuracy: 0.9793
Epoch 42, Loss: 0.0671, Accuracy: 0.9793
Epoch 43, Loss: 0.0661, Accuracy: 0.9797
Epoch 44, Loss: 0.0621, Accuracy: 0.9810
Epoch 45, Loss: 0.0617, Accuracy: 0.9811
Epoch 46, Loss: 0.0594, Accuracy: 0.9817
Epoch 47, Loss: 0.0601, Accuracy: 0.9815
Epoch 48, Loss: 0.0569, Accuracy: 0.9826
Epoch 49, Loss: 0.0585, Accuracy: 0.9824
Epoch 50, Loss: 0.0566, Accuracy: 0.9827

Test Accuracy: 0.9466, Training Time: 355.12 seconds



Training model with 64 neurons and sigmoid activation
Epoch 1, Loss: 0.4207, Accuracy: 0.8669
Epoch 2, Loss: 0.3289, Accuracy: 0.8967
Epoch 3, Loss: 0.2797, Accuracy: 0.9132
Epoch 4, Loss: 0.2467, Accuracy: 0.9241
Epoch 5, Loss: 0.2228, Accuracy: 0.9318
Epoch 6, Loss: 0.2047, Accuracy: 0.9380
Epoch 7, Loss: 0.1904, Accuracy: 0.9420
Epoch 8, Loss: 0.1783, Accuracy: 0.9458
Epoch 9, Loss: 0.1679, Accuracy: 0.9490
Epoch 10, Loss: 0.1588, Accuracy: 0.9517
Epoch 11, Loss: 0.1508, Accuracy: 0.9542
Epoch 12, Loss: 0.1436, Accuracy: 0.9564
Epoch 13, Loss: 0.1372, Accuracy: 0.9586
Epoch 14, Loss: 0.1314, Accuracy: 0.9602
Epoch 15, Loss: 0.1260, Accuracy: 0.9617
Epoch 16, Loss: 0.1210, Accuracy: 0.9633
Epoch 17, Loss: 0.1164, Accuracy: 0.9646
Epoch 18, Loss: 0.1120, Accuracy: 0.9661
Epoch 19, Loss: 0.1079, Accuracy: 0.9673
Epoch 20, Loss: 0.1040, Accuracy: 0.9687
Epoch 21, Loss: 0.1003, Accuracy: 0.9695
Epoch 22, Loss: 0.0968, Accuracy: 0.9710
Epoch 23, Loss: 0.0935, Accuracy: 0.9722
Epoch 24, Loss: 0.0903, Accuracy: 0.9733
Epoch 25, Loss: 0.0872, Accuracy: 0.9743
Epoch 26, Loss: 0.0843, Accuracy: 0.9752
Epoch 27, Loss: 0.0816, Accuracy: 0.9761
Epoch 28, Loss: 0.0790, Accuracy: 0.9768
Epoch 29, Loss: 0.0766, Accuracy: 0.9775
Epoch 30, Loss: 0.0743, Accuracy: 0.9782
Epoch 31, Loss: 0.0722, Accuracy: 0.9787
Epoch 32, Loss: 0.0701, Accuracy: 0.9794
Epoch 33, Loss: 0.0682, Accuracy: 0.9799
Epoch 34, Loss: 0.0664, Accuracy: 0.9805
Epoch 35, Loss: 0.0646, Accuracy: 0.9811
Epoch 36, Loss: 0.0630, Accuracy: 0.9816
Epoch 37, Loss: 0.0614, Accuracy: 0.9820
Epoch 38, Loss: 0.0599, Accuracy: 0.9825
Epoch 39, Loss: 0.0585, Accuracy: 0.9829
Epoch 40, Loss: 0.0570, Accuracy: 0.9834
Epoch 41, Loss: 0.0557, Accuracy: 0.9838
Epoch 42, Loss: 0.0543, Accuracy: 0.9842
Epoch 43, Loss: 0.0531, Accuracy: 0.9845
Epoch 44, Loss: 0.0518, Accuracy: 0.9849
Epoch 45, Loss: 0.0506, Accuracy: 0.9855
Epoch 46, Loss: 0.0495, Accuracy: 0.9859
Epoch 47, Loss: 0.0484, Accuracy: 0.9863
Epoch 48, Loss: 0.0473, Accuracy: 0.9866
Epoch 49, Loss: 0.0463, Accuracy: 0.9869
Epoch 50, Loss: 0.0453, Accuracy: 0.9872

Test Accuracy: 0.9547, Training Time: 378.97 seconds



Training model with 64 neurons and tanh activation
Epoch 1, Loss: 0.4040, Accuracy: 0.8758
Epoch 2, Loss: 0.3160, Accuracy: 0.9026
Epoch 3, Loss: 0.2686, Accuracy: 0.9173
Epoch 4, Loss: 0.2387, Accuracy: 0.9269
Epoch 5, Loss: 0.2176, Accuracy: 0.9329
Epoch 6, Loss: 0.2041, Accuracy: 0.9374
Epoch 7, Loss: 0.1928, Accuracy: 0.9408
Epoch 8, Loss: 0.1828, Accuracy: 0.9441
Epoch 9, Loss: 0.1743, Accuracy: 0.9466
Epoch 10, Loss: 0.1671, Accuracy: 0.9482
Epoch 11, Loss: 0.1609, Accuracy: 0.9500
Epoch 12, Loss: 0.1545, Accuracy: 0.9521
Epoch 13, Loss: 0.1476, Accuracy: 0.9540
Epoch 14, Loss: 0.1416, Accuracy: 0.9559
Epoch 15, Loss: 0.1365, Accuracy: 0.9572
Epoch 16, Loss: 0.1316, Accuracy: 0.9589
Epoch 17, Loss: 0.1277, Accuracy: 0.9600
Epoch 18, Loss: 0.1227, Accuracy: 0.9614
Epoch 19, Loss: 0.1183, Accuracy: 0.9630
Epoch 20, Loss: 0.1145, Accuracy: 0.9641
Epoch 21, Loss: 0.1109, Accuracy: 0.9651
Epoch 22, Loss: 0.1076, Accuracy: 0.9659
Epoch 23, Loss: 0.1045, Accuracy: 0.9667
Epoch 24, Loss: 0.1014, Accuracy: 0.9678
Epoch 25, Loss: 0.0988, Accuracy: 0.9685
Epoch 26, Loss: 0.0966, Accuracy: 0.9693
Epoch 27, Loss: 0.0945, Accuracy: 0.9699
Epoch 28, Loss: 0.0925, Accuracy: 0.9704
Epoch 29, Loss: 0.0907, Accuracy: 0.9713
Epoch 30, Loss: 0.0888, Accuracy: 0.9716
Epoch 31, Loss: 0.0869, Accuracy: 0.9723
Epoch 32, Loss: 0.0850, Accuracy: 0.9730
Epoch 33, Loss: 0.0832, Accuracy: 0.9737
Epoch 34, Loss: 0.0815, Accuracy: 0.9743
Epoch 35, Loss: 0.0796, Accuracy: 0.9749
Epoch 36, Loss: 0.0778, Accuracy: 0.9755
Epoch 37, Loss: 0.0762, Accuracy: 0.9760
Epoch 38, Loss: 0.0749, Accuracy: 0.9766
Epoch 39, Loss: 0.0738, Accuracy: 0.9769
Epoch 40, Loss: 0.0728, Accuracy: 0.9773
Epoch 41, Loss: 0.0714, Accuracy: 0.9776
Epoch 42, Loss: 0.0701, Accuracy: 0.9781
Epoch 43, Loss: 0.0687, Accuracy: 0.9784
Epoch 44, Loss: 0.0674, Accuracy: 0.9787
Epoch 45, Loss: 0.0660, Accuracy: 0.9790
Epoch 46, Loss: 0.0648, Accuracy: 0.9795
Epoch 47, Loss: 0.0637, Accuracy: 0.9798
Epoch 48, Loss: 0.0628, Accuracy: 0.9801
Epoch 49, Loss: 0.0617, Accuracy: 0.9805
Epoch 50, Loss: 0.0606, Accuracy: 0.9807

Test Accuracy: 0.9387, Training Time: 399.23 seconds



   Neurons Activation  Test Accuracy  Training Time (s)  \
0      256       relu         0.9603         356.808736
1      256    sigmoid         0.9566         353.331970
2      256       tanh         0.9453         346.107443
3      128       relu         0.9541         354.562662
4      128    sigmoid         0.9588         353.854586
5      128       tanh         0.9403         343.049430
6       64       relu         0.9466         355.124712
7       64    sigmoid         0.9547         378.974908
8       64       tanh         0.9387         399.227740

                                          Loss Curve  \
0  [0.41875553, 0.2500101, 0.17056994, 0.14167993...
1  [0.3540268, 0.24073473, 0.1877891, 0.1584356, ...
2  [0.50156254, 0.2969519, 0.21033926, 0.17396438...
3  [0.3932619, 0.30297422, 0.24930508, 0.2139421,...
4  [0.35397696, 0.26111376, 0.21858512, 0.1918965...
5  [0.44279176, 0.30585548, 0.23885511, 0.2079683...
6  [0.5166508, 0.36399674, 0.31308377, 0.2903138,...
7  [0.42074525, 0.32888725, 0.27972573, 0.2467252...
8  [0.40395904, 0.31595466, 0.26855624, 0.2386934...

                                      Accuracy Curve  \
0  [0.91478336, 0.94026667, 0.95465, 0.9597333, 0...
1  [0.9116167, 0.93376666, 0.9457167, 0.9519333, ...
2  [0.9081333, 0.9316667, 0.945, 0.9511167, 0.953...
3  [0.89708334, 0.91675, 0.92796665, 0.9369, 0.94...
4  [0.8982667, 0.92181665, 0.93476665, 0.94263333...
5  [0.87733334, 0.9087333, 0.9273, 0.9369, 0.9445...
6  [0.8605, 0.9008167, 0.9135, 0.9198, 0.92876667...
7  [0.8669, 0.89666665, 0.9132, 0.92408335, 0.931...
8  [0.8757667, 0.9026333, 0.91726667, 0.92693335,...

                                    Confusion Matrix
0  [[962, 0, 3, 0, 2, 4, 2, 3, 1, 3], [1, 1119, 5...
1  [[963, 0, 1, 1, 1, 4, 4, 3, 2, 1], [0, 1126, 1...
2  [[940, 0, 4, 1, 3, 11, 10, 1, 3, 7], [0, 1115,...
3  [[955, 1, 3, 0, 1, 5, 7, 2, 3, 3], [0, 1120, 3...
4  [[966, 1, 3, 0, 0, 4, 2, 2, 2, 0], [0, 1120, 2...
5  [[952, 0, 5, 2, 2, 11, 4, 2, 2, 0], [0, 1110, ...
6  [[963, 1, 3, 1, 0, 4, 2, 3, 3, 0], [1, 1114, 4...
7  [[962, 1, 1, 2, 0, 6, 2, 1, 1, 4], [0, 1120, 4...
8  [[957, 0, 5, 1, 2, 6, 4, 2, 0, 3], [0, 1108, 4...








